# 중고차 사이트 웹크롤링
  
* 차량명/인승(숫자)/마력(숫자)/연식/연료/주행/가격  
* 데이터프레임으로 표현

In [77]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import tqdm
import pandas as pd
from tqdm import tqdm_notebook

In [43]:
driver = webdriver.Chrome("C://mychrome/chromedriver.exe")

In [83]:
page = int(input("검색 범위의 최종 페이지를 입력하세요. : "))
total_product_list = []
for p in tqdm.notebook.tqdm(range(1, page+1)):
    url = "https://www.bobaedream.co.kr/cyber/CyberCar.php?gubun=K&page={}&order=S11&view_size=50".format(p)
    driver.get(url)
    time.sleep(3)


    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    products = soup.select("li.product-item")
    prod_list = []
    for prod in products:
        title = prod.select("div.mode-cell.title > p.tit.ellipsis > a")[0].text.strip()
        prod_spec = prod.select("dl.data.clearfix > dd.data-item")
        num_p = None
        ma = None
        for spec in prod_spec:
            if spec.select("a") and '인승' in spec.select("a")[0].text:
                num_p = spec.select("a")[0].text.split("인승")[0]
            elif '마력' in spec.text:
                ma = spec.text
        year = prod.select("div.mode-cell.year > span")[0].text
        if "(" in year:
            year = year.split("(")[0]
        fuel = prod.select("div.mode-cell.fuel > span")[0].text
        km = prod.select("div.mode-cell.km > span")[0].text 
        price = None
        if prod.select("div.mode-cell.price > strong"):
            price = prod.select("div.mode-cell.price > strong")[0].text


        prod_list.append([title, num_p, ma, year, fuel, km, price])
    total_product_list += prod_list

검색 범위의 최종 페이지를 입력하세요. : 2


In [87]:
print("총 {}개의 데이터가 있습니다.".format(len(total_product_list)))
df = pd.DataFrame(total_product_list)
df.columns = ['차량명', '인승', '마력', '연식', '연료', '주행', '가격']
df

총 100개의 데이터가 있습니다.


,차량명,인승,마력,연식,연료,주행,가격
0,쉐보레 스파크 LT,5,70마력,13/10,가솔린,3만km,290
1,현대 제네시스 쿠페 380 GT R,4,303마력,10/04,가솔린,11만km,690
2,맥스 카니발 리무진,9,280마력,16/12,가솔린,5만km,"3,490"
3,현대 그랜져IG 2.2 e-VGT 프리미엄 스페셜,5,202마력,17/11,디젤,3만km,"3,150"
4,기아 더 프레스티지 K7 3.0 LPi 프레스티지,5,235마력,12/03,LPG,14만km,"1,000"
...,...,...,...,...,...,...,...
95,현대 그레이스 LPG 15인승 그랜드 살롱,15,None,02/03,LPG,13만km,None
96,현대 아반떼MD M16 GDi 스마트,5,140마력,13/04,가솔린,4만km,700
97,기아 올 뉴 카니발 2.2 디젤 7인승 리무진 VIP,7,202마력,15/09,디젤,13만km,"2,290"
98,기아 더 뉴 스포티지R 2.0 T-GDI 트렌디,5,261마력,13/12,가솔린,8만km,"1,390"
